<a href="https://colab.research.google.com/github/AndresMontesDeOca/TextMining/blob/main/Text_Mining_CGC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

#### Universidad Austral<br>
#### Maestría en Minería de Datos y Gestión del Conocimiento<br>
#### Text Mining<br>
#### CGC: Clasificador de Géneros Cinematográficos<br>


#### Integrantes:<br>
Alejandra Reyes<br>
Andres Montes de Oca<br>
Rafael Gimenez<br>
Soledad Ríos<br>
Tomas Sauro

</center>

## Introducción
Título: Clasificador de Géneros Cinematográficos (CGC)

Problemática: En un servicio que releva los contenidos disponibles en plataformas streaming, con frecuencuencia se detectan contenidos sin información de género pero se cuenta con la sinopsis del contenido.

Nombre del servicio y empresa con la problematica: [Content Pulse | BB Media](https://bb.vision/content-pulse-en/)

Objetivo: Entrenar un modelo que dada una sinopsis sobre una película retorne los géneros del contenido.

Hoja de Ruta:

1. Acceso a los datos
2. Tabulación de los datos
3. Limpieza de datos
4. Exploración de modelos
5. Pruebas
6. Analísis de resultados
7. Conclusiones

Posibles contratiempos:

1. Datos faltantes, incompletos, no estandarizados.
2. Fallar en una buena separacion de los subdataset para testing y validation (ejemplo, quedarnos con sinopsis para testing muy ricas en descripción y para validation no).

## 1. Acceso a los datos


Se utiliza una base de datos colaborativas con acceso abierto llamada TMDB obtenido los datos mediante su [API](https://developer.themoviedb.org/docs)

In [ ]:
import requests
import pandas as pd
import json
import datetime

# Credencial
api_key = 'bf0a945ba271caf72a6a1b1f53a1084d'

# URL base
base_url = 'https://api.themoviedb.org/3/'

# Endpoint para obtener la lista de películas y series de tv
movie_endpoint = 'discover/movie'
tv_endpoint = 'discover/tv'

# Parámetros
params = {
    'api_key': api_key,
    'sort_by': 'popularity.desc' # Se usa este orden para verificar si los datos obtenidos son correctos (los contenidos mas populares son mas conocidos siendo mas facil validar)
}

# Función para obtener nombres de actores
def get_cast_names(content_type, content_id):
    credits_endpoint = f'{content_type}/{content_id}/credits'
    credits_params = {
        'api_key': api_key
    }
    credits_response = requests.get(base_url + credits_endpoint, params=credits_params)
    credits_data = credits_response.json()
    cast_names = [actor['name'] for actor in credits_data['cast'][:3]] # Dado que los actores estan ordenados por importancia, nos quedamos con los 3 primeros (protagonistas)
    return ', '.join(cast_names)

# Función para obtener nombres de compañías de producción
def get_production_companies(content_type, content_id):
    details_endpoint = f'{content_type}/{content_id}'
    details_params = {
        'api_key': api_key
    }
    details_response = requests.get(base_url + details_endpoint, params=details_params)
    details_data = details_response.json()
    production_companies = ', '.join([company['name'] for company in details_data.get('production_companies', [])])
    return production_companies

# Realizar solicitudes GET a la API y obtener datos para películas
movie_response = requests.get(base_url + movie_endpoint, params=params)
movie_data = movie_response.json()

# Realizar solicitudes GET a la API y obtener datos para programas de televisión
tv_response = requests.get(base_url + tv_endpoint, params=params)
tv_data = tv_response.json()

# Procesar los resultados de películas y programas de televisión
results_list = []

for content_type, content_data in [('movie', movie_data), ('tv', tv_data)]:
    for result in content_data['results']:
        content_id = result['id']
        title = result['title'] if content_type == 'movie' else result['name']
        release_date = result['release_date'] if content_type == 'movie' else result['first_air_date']
        overview = result['overview']
        poster_path = result['poster_path']
        popularity = result['popularity']
        vote_count = result['vote_count']
        vote_average = result['vote_average']

        # Obtener nombres de actores utilizando la función
        cast_names = get_cast_names(content_type, content_id)

        # Obtener nombres de compañías de producción utilizando la nueva función
        production_companies = get_production_companies(content_type, content_id)

        results_list.append({
            'type': content_type.capitalize(),
            'id': content_id,
            'title': title,
            'release': release_date,
            'synopsis': overview,
            'cast': cast_names,
            'productions': production_companies,
            'popularity': popularity,
            'votes': vote_count,
            'score': vote_average,
            'imagen': f'https://image.tmdb.org/t/p/w500{poster_path}'
        })

# DataFrame
tmdb_base_20 = pd.DataFrame(results_list)
tmdb_base_20.head()

,type,id,title,release,synopsis,cast,productions,popularity,votes,score,imagen
0,Movie,976573,Elemental,2023-06-14,"In a city where fire, water, land and air resi...","Leah Lewis, Mamoudou Athie, Ronnie del Carmen","Walt Disney Pictures, Pixar",3832.175,1360,7.8,https://image.tmdb.org/t/p/w500/6oH378KUfCEitz...
1,Movie,724209,Heart of Stone,2023-08-09,An intelligence operative for a shadowy global...,"Gal Gadot, Jamie Dornan, Alia Bhatt","Skydance, Pilot Wave Motion Pictures, Mockingb...",3796.765,734,7.0,https://image.tmdb.org/t/p/w500/vB8o2p4ETnrfiW...
2,Movie,569094,Spider-Man: Across the Spider-Verse,2023-05-31,"After reuniting with Gwen Stacy, Brooklyn’s fu...","Shameik Moore, Hailee Steinfeld, Jason Schwart...","Columbia Pictures, Sony Pictures Animation, Lo...",2433.631,3722,8.5,https://image.tmdb.org/t/p/w500/8Vt6mWEReuy4Of...
3,Movie,667538,Transformers: Rise of the Beasts,2023-06-06,When a new threat capable of destroying the en...,"Anthony Ramos, Dominique Fishback, Peter Cullen","Skydance, Paramount, di Bonaventura Pictures, ...",1596.732,2915,7.5,https://image.tmdb.org/t/p/w500/gPbM0MK8CP8A17...
4,Movie,298618,The Flash,2023-06-13,When his attempt to save his family inadverten...,"Ezra Miller, Sasha Calle, Michael Keaton","Warner Bros. Pictures, Double Dream, The Disco...",1405.843,2463,7.0,https://image.tmdb.org/t/p/w500/rktDFPbfHfUbAr...


In [ ]:
count_by_type_20 = tmdb_base_20['type'].value_counts()
count_by_type_20

Movie    20
Tv       20
Name: type, dtype: int64

In [ ]:
import requests
import pandas as pd
import json
import datetime

# Credencial
api_key = 'bf0a945ba271caf72a6a1b1f53a1084d'

# URL base
base_url = 'https://api.themoviedb.org/3/'

# Endpoint para obtener la lista de películas y series de tv
movie_endpoint = 'discover/movie'
tv_endpoint = 'discover/tv'

# Parámetros comunes
common_params = {
    'api_key': api_key,
    'sort_by': 'popularity.desc',
    'page': 1  # Comenzamos desde la primera página
}

def get_all_results(endpoint):
    all_results = []
    page = 1
    while True:
        response = requests.get(base_url + endpoint, params={**common_params, 'page': page})
        data = response.json()
        results = data
        if not results:
            break
        all_results.extend(results)
        page += 1
    return all_results

# Obtener todas las películas y series
all_movies = get_all_results(movie_endpoint)
all_series = get_all_results(tv_endpoint)

# DataFrames con todos los id de peliculas y series
movies_df = pd.DataFrame(all_movies)
series_df = pd.DataFrame(all_series)

In [ ]:
# Función para obtener nombres de actores
def get_cast_names(content_type, content_id):
    credits_endpoint = f'{content_type}/{content_id}/credits'
    credits_params = {
        'api_key': api_key
    }
    credits_response = requests.get(base_url + credits_endpoint, params=credits_params)
    credits_data = credits_response.json()
    cast_names = [actor['name'] for actor in credits_data['cast'][:3]]
    return ', '.join(cast_names)

# Función para obtener nombres de compañías de producción
def get_production_companies(content_type, content_id):
    details_endpoint = f'{content_type}/{content_id}'
    details_params = {
        'api_key': api_key
    }
    details_response = requests.get(base_url + details_endpoint, params=details_params)
    details_data = details_response.json()
    production_companies = ', '.join([company['name'] for company in details_data.get('production_companies', [])])
    return production_companies

# Procesar los resultados de películas y programas de televisión
results_list = []

for content_type, content_data in [('movie', movies_df.iterrows()), ('tv', series_df.iterrows())]:
    for _, result in content_data:
        content_id = result['id']
        title = result['title'] if content_type == 'movie' else result['name']
        release_date = result['release_date'] if content_type == 'movie' else result['first_air_date']
        overview = result['overview']
        poster_path = result['poster_path']
        popularity = result['popularity']
        vote_count = result['vote_count']
        vote_average = result['vote_average']

        # Obtener nombres de actores utilizando la función
        cast_names = get_cast_names(content_type, content_id)

        # Obtener nombres de compañías de producción utilizando la nueva función
        production_companies = get_production_companies(content_type, content_id)

        results_list.append({
            'type': content_type.capitalize(),
            'id': content_id,
            'title': title,
            'release': release_date,
            'synopsis': overview,
            'cast': cast_names,
            'productions': production_companies,
            'popularity': popularity,
            'votes': vote_count,
            'score': vote_average,
            'imagen': f'https://image.tmdb.org/t/p/w500{poster_path}'
        })

# DataFrame Final
tmdb_base = pd.DataFrame(results_list)
tmdb_base.head()

In [ ]:
count_by_type = tmdb_base['type'].value_counts()
count_by_type

In [ ]:
# Obtener la fecha actual para controlar el archivo que se exporta
current_date = datetime.datetime.now().strftime("%Y-%m-%d")

# CSV
csv_filename = f'tmdb_data_{current_date}.csv'
tmdb_base.to_csv(csv_filename, index=False)